In [57]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
# %matplotlib qt para graficos afuera. Por defecto es inline dentro de la notebook

df_test = pd.read_csv("alturas-pesos-mils-test.csv")
df_train = pd.read_csv("alturas-pesos-mils-train.csv")
df_train.drop(columns=["Unnamed: 0"],inplace=True)
df_test.drop(columns=["Unnamed: 0"],inplace=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df_train2, df_valid = train_test_split(df_train, test_size=0.30)

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 3 columns):
Genero    8000 non-null object
Peso      8000 non-null float64
Altura    8000 non-null float64
dtypes: float64(2), object(1)
memory usage: 187.6+ KB


In [7]:
len(df_train2)

5600

In [5]:
df_train2[df_train2["Genero"]=="Mujer"].describe()

,Peso,Altura
count,2808.000000,2808.000000
mean,61.194073,161.290042
std,8.594072,6.830923
min,31.783000,138.866000
25%,55.416000,156.699750
50%,61.232500,161.320500
75%,66.933000,165.876250
max,91.056000,186.469000


In [6]:
df_train2[df_train2["Genero"]=="Hombre"].describe()

,Peso,Altura
count,2792.000000,2792.000000
mean,84.462753,174.920977
std,8.848514,7.205591
min,50.694000,147.908000
25%,78.430500,170.268000
50%,84.541500,174.993500
75%,90.466500,179.938000
max,121.687000,199.824000


In [86]:
#np.histogram2d(df_train2[df_train2["Genero"]=="Mujer"]["Peso"].to_numpy(),
#               df_train2[df_train2["Genero"]=="Mujer"]["Altura"].to_numpy(),
#              bins=30)

def eval_model(df_train, df_valid, pbins, abins):
    x = df_train[df_train["Genero"]=="Mujer"]["Peso"].to_numpy()
    y = df_train[df_train["Genero"]=="Mujer"]["Altura"].to_numpy()
    histo_m = np.histogram2d(x, y, bins=[pbins,abins])
    x = df_train[df_train["Genero"]=="Hombre"]["Peso"].to_numpy()
    y = df_train[df_train["Genero"]=="Hombre"]["Altura"].to_numpy()
    histo_h = np.histogram2d(x, y, bins=[pbins,abins])
                     #range=[[df_train["Peso"].min(),df_train["Peso"].max()],
                     #      [df_train["Altura"].min(),df_train["Altura"].max()]])
    arWeight_m = np.digitize(df_valid["Peso"].values,histo_m[1])-1
    arWeight_m[arWeight_m==-1]=0
    arWeight_m[arWeight_m>=pbins]=pbins-1

    arHeight_m = np.digitize(df_valid["Altura"].values,histo_m[2])-1
    arHeight_m[arHeight_m==-1]=0
    arHeight_m[arHeight_m>=abins]=abins-1

    arWeight_h = np.digitize(df_valid["Peso"].values,histo_h[1])-1
    arWeight_h[arWeight_h==-1]=0
    arWeight_h[arWeight_h>=pbins]=pbins-1

    arHeight_h = np.digitize(df_valid["Altura"].values,histo_h[2])-1
    arHeight_h[arHeight_h==-1]=0
    arHeight_h[arHeight_h>=abins]=abins-1
    
    pred = histo_m[0][list(arWeight_m),list(arHeight_m)] > histo_h[0][list(arWeight_h),list(arHeight_h)]
    
    pred = pred == (df_valid["Genero"]=="Mujer")
    
    return sum(pred)/len(pred)

In [87]:
print("bins=[{},{}] accuracy= {}".format(i,j,eval_model(df_train2,df_valid,23,23)))

bins=[69,69] accuracy= 0.9058333333333334


In [91]:
%%time
#df = pd.DataFrame(columns=[{'pbin'},{'abin'},{'accur'}])

acc = 0
for i in range(1,70):
    for j in range(1,70):
        acc_aux = eval_model(df_train2,df_valid,i,j)
        if acc_aux> acc:
            acc = acc_aux
            best_binp = i
            best_bina = j
        

CPU times: user 56.1 s, sys: 15.7 ms, total: 56.1 s
Wall time: 56.1 s


In [92]:
print(best_binp,best_bina,acc)

26 18 0.9108333333333334
